# Preprocessing with Fugue

In the first section, we saw how to scale `StatsForecast` on top of Spark, Dask, and Ray just by changing one line of code. When dealing with large data, we'll need to preprocess it also to get it in the same format. How do we prototype on small data, and scale it seamlessly? [Fugue](https://github.com/fugue-project/fugue/) allows us to scale existing Python and Pandas code to Spark, Dask, or Ray just by changing the engine as well.

In this section, we'll preprocess the data shown in the previous section using Dask to speed it up. One of the advantages of Fugue is being able to decouple logic and execution. Code can be prototyped on the local system, and brought to the full scale when ready.

Below is an image of the architecture of Fugue. It's an abstraction layer on top of the distributed computing frameworks.

![architecture](../img/architecture.png)

## Loading in Data

We'll start with pre-processing the `training_data` that contains the sales volume. The problem with this file is that it is very wide with around 2000 columns, but `StatsForecast` expects data in a long format where each data point is one row. Though this is a memory efficient way to store the data, it's unfriendly to work with for machine learning. We need to transform the data first.

In [1]:
import pandas as pd
import os

download_path = os.path.abspath(os.path.join(".","..","data","m5-forecasting-accuracy.zip"))
unzipped_path = os.path.abspath(os.path.join(".","..","data","m5-forecasting-accuracy-unzipped"))

# Read in the data
INPUT_DIR = unzipped_path
WORKING_DIR = os.path.join(unzipped_path, "..", "working")
training_data = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')


## Processing Training Data

Recall the first row of the DataFrame. We have the days as column names and and the time series for one `id` is contained in the row.

In [2]:
training_data.iloc[0:1]

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1


We will need the `calendar` data to align to the `training_data`. We'll display the head also just to remember what the data format looks like.

In [3]:
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
calendar["date"] = pd.to_datetime(calendar["date"])
calendar.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


Below is a function to pivot the timeseries information into a long format. Note that this function does not use Pandas. It runs on each row of data in the `training_data` DataFrame and creates a new day entry. While this logic can be done in Pandas, one of the advantages of using Fugue is not everything has to be expressed in Pandas. Some opeartions will be clearer in native Python. Fugue will apply the conversions to run this on top of Pandas, Spark, Dask, or Ray.

In the `format_sales` function, we treat each function as a row and loop through the rows of the DataFrame. For each row, we pull out each day and value, and then `yield` a new row. Adding `0.01` to the timeseries values is a technique to improve numerical stability when dealing with timeseries data. If this function is not clear, the results below will make it clearer when we apply it on the first row of data.

In [4]:
from typing import Iterable, List, Any, Dict
from datetime import timedelta

start = calendar['date'].min()

def format_sales(df:Iterable[List[Any]], start) -> Iterable[List[Any]]:
    for row in df:
        counter = 0
        for y in row[6:]:
            # help with convergence
            if y == 0:
                y = y + 0.01
            date = start + timedelta(counter)
            yield row[:6] + [date, y]
            counter=counter+1

Now we use Fugue to run it on the first row of data from our `training_data` DataFrame. The `transform()` function of Fugue is used to distribute one step of Python or Pandas. As long as the implementation on Pandas works, it will also work on Spark, Dask, and Ray.

Schema is a requirement for the distributed backends, so it's also required when applying function on top of Pandas. It's also needed to apply column names to the `List` output. The Fugue schema expression will be more minimal compared to defining it directly on the backend.

We don't necessarily need all of the hierarchichal information, but they will be useful in the next section when we apply hierarchichal forecasting.

In [5]:
from fugue import transform

transform(training_data.iloc[0:1], 
          format_sales, 
          schema="unique_id:str,item_id:str,dept_id:str,cat_id:str,store_id:str,state_id:str,ds:date,y:float",
          params={"start": start})

,unique_id,item_id,dept_id,cat_id,store_id,state_id,ds,y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,0.01
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-30,0.01
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-31,0.01
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-02-01,0.01
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-02-02,0.01
...,...,...,...,...,...,...,...,...
1936,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-18,0.01
1937,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-19,3.00
1938,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-20,3.00
1939,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2016-05-21,0.01


When run on the first row, it expanded the data out to one row per day. Because it worked when we tested it quickly, we can now run it on top of Spark, Dask, or Ray by changing the engine. In the example below, we run it on top of Dask. We can demo this on the first 100 rows of data.

As mentioned in the first notebook, we need to call `.compute()` in order to trigger the execution of the Dask DataFrame.

In [6]:
ddf = transform(training_data[0:100], 
                format_sales, 
                schema="unique_id:str,item_id:str,dept_id:str,cat_id:str,store_id:str,state_id:str,ds:date,y:float",
                params={"start": start},
                engine="dask")
ddf.compute().head(5)

/opt/anaconda3/envs/pydata/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49193 instead
  warnings.warn(
2023-02-13 00:35:33,561 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/w2/91_v34nx0xs2npnl3zsl9tmm0000gn/T/dask-worker-space/worker-j1o9lf_1', purging
2023-02-13 00:35:33,562 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/w2/91_v34nx0xs2npnl3zsl9tmm0000gn/T/dask-worker-space/worker-jg_tza5a', purging
2023-02-13 00:35:33,564 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/w2/91_v34nx0xs2npnl3zsl9tmm0000gn/T/dask-worker-space/worker-1qr0fzd8', purging
2023-02-13 00:35:33,565 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/w2/91_v34nx0xs2npnl3zsl9tmm0000gn/T/dask-worker-space/worker-69row5ay', purging
2023-02-13 00:35:33,565

,unique_id,item_id,dept_id,cat_id,store_id,state_id,ds,y
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-29,0.01
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-30,0.01
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-01-31,0.01
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-02-01,0.01
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2011-02-02,0.01


## Exogenous Regressors

The format above is very close to the format `StatsForecast` expects. We basically already have `unique_id`, `ds`, and `y`. But for this scenario, we want to add price in as an exogenous regressor. Models that can handle exogenous regressors will utilize it, while the ones that can't still just ignore it.

Below we load the `sell_prices`.

In [7]:
sell_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
sell_prices.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


In [8]:
calendar.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


## Combining All Data

In order to combine everything together, we just need to use the `transform()` function earlier on the sales data and then join the other tables. We will sample the `training_data` frame just to test all the joins work before we run it on the full data.

For joins, Fugue also has `join` functions that work for Pandas, Spark, Dask, and Ray DataFrames. Here, we'll see how to use it. Again, the advantage is that we don't need to write separate code for both small and large scale data.

In [9]:
import fugue.api as fa 

sample = training_data.iloc[0:2].copy()
long = transform(sample, 
                format_sales, 
                schema="unique_id:str,item_id:str,dept_id:str,cat_id:str,store_id:str,state_id:str,ds:date,y:float",
                params={"start": start},
                engine="dask")

# rename the column date to ds
calendar_tmp = fa.rename(calendar.copy(), {"date": "ds"})
combined = fa.join(long, calendar_tmp[["ds","wm_yr_wk"]], how="left_outer")
combined = fa.join(combined, sell_prices, how="inner")
combined.head()

,unique_id,item_id,dept_id,cat_id,store_id,state_id,ds,y,wm_yr_wk,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2013-10-01,2.00,11336,8.26
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2013-10-04,0.01,11336,8.26
2,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2013-09-29,0.01,11336,8.26
3,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2013-10-02,0.01,11336,8.26
4,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,2013-09-28,0.01,11336,8.26


The `rename` function is used to rename the columns of a DataFrame. Now, we can use exactly the same code to run everything on Dask by using the Fugue functions under the Fugue `engine_context()`. We don't need to specify the `engine` under `transform()` anymore because Fugue will know to use Dask.

This cell may have a lot of logs about garbage collection warnings when being run on a local machine. This is we will have a number of Dask tasks compared to our resources. When running on a cluster with more compute, this will be less of a problem. It may also take a while to finish (somewhere like 10 minutes).

In [10]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)

with fa.engine_context(client):
    long = transform(training_data, 
                    format_sales, 
                    schema="unique_id:str,item_id:str,dept_id:str,cat_id:str,store_id:str,state_id:str,ds:date,y:float",
                    params={"start": start})

    # rename the column date to ds
    calendar_tmp = fa.rename(calendar.copy(), {"date": "ds"})
    combined = fa.join(long, calendar_tmp[["ds","wm_yr_wk"]], how="left_outer")
    combined = fa.join(combined, sell_prices, how="inner")
    fa.save(combined, f"{WORKING_DIR}/combined.parquet")

/opt/anaconda3/envs/pydata/lib/python3.8/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49240 instead
  warnings.warn(
2023-02-13 00:37:14,230 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.51 GiB -- Worker memory limit: 4.00 GiB
2023-02-13 00:37:14,616 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.62 GiB -- Worker memory limit: 4.00 GiB
2023-02-13 00:37:16,815 - distributed.worker.memory - WARNING - U

## Example Regression with Exogenous Regressors

This is an example of how to run a regression using the exogenous regressors. The important part is that the exogenous regressions need to be available for the forecast horizon.

We will just do this on Pandas to quickly demo, but it can be scale on the distributed backends as well using the `FugueBackend()`.

In [11]:
from statsforecast.models import AutoARIMA
from statsforecast import StatsForecast

sample = training_data.iloc[0:1].copy()
long = transform(sample, 
                format_sales, 
                schema="unique_id:str,item_id:str,dept_id:str,cat_id:str,store_id:str,state_id:str,ds:date,y:float",
                params={"start": start})
calendar_tmp = fa.rename(calendar.copy(), {"date": "ds"})
combined_exo = fa.join(long, calendar_tmp[["ds","wm_yr_wk"]], how="left_outer")
combined_exo = fa.join(combined_exo, sell_prices, how="inner")
combined_exo = combined_exo[['unique_id', 'ds', 'y', 'sell_price']]

/opt/anaconda3/envs/pydata/lib/python3.8/site-packages/statsforecast/core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [12]:
model = StatsForecast(
    df=combined_exo,
    models=[AutoARIMA(season_length=7)], 
    freq='D', 
    n_jobs=-1
)

# We can forecast for one timeseries. 
# We need the future values of exogenous regressors.
sample = pd.DataFrame({
    'unique_id': ['HOBBIES_1_002_CA_1_evaluation']*7,
    'ds': pd.date_range(start='2016-05-22', end='2016-05-28').tolist(),
    'sell_price': [3.97] * 7
})
# Pass a forecast to the forecast
model.forecast(7, X_df=sample)

,ds,AutoARIMA
unique_id,,
HOBBIES_1_001_CA_1_evaluation,2016-05-23,1.117753
HOBBIES_1_001_CA_1_evaluation,2016-05-24,1.117753
HOBBIES_1_001_CA_1_evaluation,2016-05-25,1.117753
HOBBIES_1_001_CA_1_evaluation,2016-05-26,1.120463
HOBBIES_1_001_CA_1_evaluation,2016-05-27,1.120463
HOBBIES_1_001_CA_1_evaluation,2016-05-28,1.117753
HOBBIES_1_001_CA_1_evaluation,2016-05-29,1.118650


## Next Steps

In the next two sections, we'll explore two different topics. The first one is Hierarchical Forecasting using Nixtla's [hierarchicalforecast](https://github.com/Nixtla/hierarchicalforecast) library. In the second one, we look at how to scale this over a cluster using Coiled to get managed Dask clusters.